<a href="https://colab.research.google.com/github/Arthy2005/NanoGPT_built_GPT-style_Transformer_from_scratch/blob/main/gpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2026-01-08 14:54:05--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt.1’

input.txt.1         100%[===================>]   1.06M  --.-KB/s    in 0.04s   

2026-01-08 14:54:06 (30.2 MB/s) - ‘input.txt.1’ saved [1115394/1115394]



In [ ]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [ ]:
print(len(text)) # 1,115,394 characters

1115394


In [ ]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [ ]:
# converts text into a set like {b, a, n} for set("banana") which will get converted into list then into a sorted list
print(set("banana"))

{'a', 'b', 'n'}


In [ ]:
characters = sorted(list(set(text)))  # converts text into a set like {b, a, n} for set("banana") which will get converted into list then into a sorted list
print(''.join(characters))
print(len(characters))


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


We need an encoder/decoder in order to encode text into integers and vise versa

In [ ]:
string_to_integer = {ch:i for i,ch in enumerate(characters)}  # characters: corresponding integer/index  pairs for all characters
integer_to_string = {i:ch for i,ch in enumerate(characters)}  #  integer/index: corresponding characters  pairs for all characters

def encode(string):
  list = []
  for char in string:
    list.append(string_to_integer[char])
  return list

def decode(list):
  string = ''
  for element in list:
    string = string + (integer_to_string[element])
  return string

print(encode("hii there"))
print(decode(encode("hii there")))


[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [ ]:
  # After creating the encoder/decoder now we have to encode our text
import torch

data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000])


torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [ ]:
# Splitting our data for training and validation 90/10 % split
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

A Transformer is a neural network architecture designed to understand sequences (text, code, DNA, audio).

Instead of reading text one word at a time (like old RNNs), it looks at many tokens at once and decides what is important using attention.

Thats why it becomes necessary to limit text to a block/chunk which we will use for training transformer.

In [ ]:
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

What we want to do is as follows:

  In the context of 18 -  47 comes next.

  In the context of 18, 47 - 56 comes next.

  In the context of... etc,
  
  This is because we want the transformer to see the patterns using a token upto a block_size.

In [ ]:
x = train_data[:block_size]
y = train_data[:block_size+1]
for t in range(block_size):
  print(f"In the context of {x[:t+1]} - {y[t+1]} comes next ")


In the context of tensor([18]) - 47 comes next 
In the context of tensor([18, 47]) - 56 comes next 
In the context of tensor([18, 47, 56]) - 57 comes next 
In the context of tensor([18, 47, 56, 57]) - 58 comes next 
In the context of tensor([18, 47, 56, 57, 58]) - 1 comes next 
In the context of tensor([18, 47, 56, 57, 58,  1]) - 15 comes next 
In the context of tensor([18, 47, 56, 57, 58,  1, 15]) - 47 comes next 
In the context of tensor([18, 47, 56, 57, 58,  1, 15, 47]) - 58 comes next 


Time dimension : In sequence modeling, Tokens are treated as events happening overtime, even if data is text.

 Each token = One time step

 8 tokens = 8 steps in time

  Time Dimension = Block Size

Since GPUs have to power to process data parallely , we can train on multiple chunks at the same time. This adds an extra dimension which is batch.

In [ ]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split):
  if split == 'train':
    data = train_data
  else:
    data = val_data
  rand_start_pos = torch.randint(len(data) - block_size, (batch_size,)) #This create batch_size amount of random starting positions in our data for processing at the same time

  x = torch.stack([data[i:i+block_size] for i in rand_start_pos])
  y = torch.stack([data[i+1:i+block_size+1] for i in rand_start_pos])
  return x, y


x_inputs, y_targets = get_batch('train')
print('inputs:')
print(x_inputs)
print('targets:')
print(y_targets)


inputs:
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])


len(data) - block_size  ensures we don't go out of bounds when slicing

1 row contains 8 examples due to the time dimensions, so 4 rows contains 32 examples for the transformer

In [ ]:
# Version 1 of Biagram Language Model
import torch.nn as nn
from torch.nn import functional as F

torch.manual_seed(1337)
vocab_size = len(characters)

class BigramLanguageModel(nn.Module):

  def __init__(self, vocab_size):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size, vocab_size) # a table of curr_char(a) and next char([2.1,-1.3,0.5.......,2.2])  where each row says If I see this character, how likely is every possible next character?

  def forward(self, idx, targets):
    # idx(indices of token ) and targets are both (B mens batch, T means time) tensors of integers
    logits = self.token_embedding_table(idx) # logits is Raw prediction scores for each possible next token

    return logits
m = BigramLanguageModel(vocab_size)
out = m(x_inputs, y_targets)
print(out.shape)

torch.Size([4, 8, 65])


Above 65 is channel dimension , what happens is each integer in our x_inputs for example lets take 5 , will go to our token_embedding_table and pluck out row 5 which will have 65 values corresponding to probability of next character/token corresponding to 5 , this will happen for all integers and a new tensor of [B, T, C] is formed.

In Biagram Language Model the next character depends only on the current character.

In [ ]:
torch.manual_seed(1337)
vocab_size = len(characters)

class BigramLanguageModel(nn.Module):

  def __init__(self, vocab_size):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size, vocab_size) # a table of curr_char(a) and next char([2.1,-1.3,0.5.......,2.2])  where each row says If I see this character, how likely is every possible next character?

  def forward(self, idx, targets):
    # idx(indices of token ) and targets are both (B mens batch, T means time) tensors of integers
    logits = self.token_embedding_table(idx) # logits is Raw prediction scores for each possible next token

    B, T, C = logits.shape
    logits = logits.view(B*T, C) # Stretching the 3D tensor into a 2D one
    targets = targets.view(B*T)
    loss = F.cross_entropy(logits, targets) # Pytorch need Channel dimension to be the second arguments, thus we had to do logits.view(B*T,C)

    return logits, loss


m = BigramLanguageModel(vocab_size)
logits, loss = m(x_inputs, y_targets)
print(logits.shape)
print(loss)

torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)


negative log likelihood

-ln(1/65) = 4.174


In [ ]:
torch.manual_seed(1337)
vocab_size = len(characters)

class BigramLanguageModel(nn.Module):

  def __init__(self, vocab_size):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size, vocab_size) # a table of curr_char(a) and next char([2.1,-1.3,0.5.......,2.2])  where each row says If I see this character, how likely is every possible next character?

  def forward(self, idx, targets=None):
    # idx(indices of token ) and targets are both (B mens batch, T means time) tensors of integers
    logits = self.token_embedding_table(idx) # logits is Raw prediction scores for each possible next token
    if targets is None:
      loss = None
    else:
      B, T, C = logits.shape
      logits = logits.view(B*T, C) # Stretching the 3D tensor into a 2D one
      targets = targets.view(B*T)
      loss = F.cross_entropy(logits, targets) # Pytorch need Channel dimension to be the second arguments, thus we had to do logits.view(B*T,C)

    return logits, loss
# Write new text, one token at a time
  def generate(self, idx, max_new_tokens):
    # idx is (B, T) array of indices in the current context
    for _ in range(max_new_tokens): # becomes (B, C)
      logits, loss = self(idx) # Gets predictions for every token position
      logits = logits[:,-1,:]
      probs = F.softmax(logits, dim=-1) # Converts raw scores(logits) into probablities # (B, C)
      idx_next = torch.multinomial(probs, num_samples=1) # Randomly choose/samples the next token on probabilities # (B, 1)
      idx = torch.cat((idx, idx_next), dim=1)#(B, T+1)
    return idx


m = BigramLanguageModel(vocab_size)
logits, loss = m(x_inputs, y_targets)
print(logits.shape)
print(loss)
print(decode(m.generate(torch.zeros((1,1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)

SKIcLT;AcELMoTbvZv C?nq-QE33:CJqkOKH-q;:la!oiywkHjgChzbQ?u!3bLIgwevmyFJGUGp
wnYWmnxKWWev-tDqXErVKLgJ


The above is not trained yet, it works on random values and since its a bigram language model it only use the last character for guess.

In [ ]:
# create a Pytorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [ ]:
batch_size = 32
for steps in range(10000):
  x_inputs, y_targets = get_batch('train') #choose/sample a batch of data
  logits, loss = m(x_inputs, y_targets) # evaluate the loss
  optimizer.zero_grad(set_to_none=True) #pytorch accumulates gradients so old gradients must be cleared before computing new ones
  loss.backward() #backpropogation
  optimizer.step()#uses gradient to move towards lower loss

print(loss.item())

2.382369041442871


In [ ]:
print(decode(m.generate(torch.zeros((1,1), dtype=torch.long), max_new_tokens=300)[0].tolist()))


lso br. ave aviasurf my, yxMPZI ivee iuedrd whar ksth y h bora s be hese, woweee; the! KI 'de, ulseecherd d o blllando;LUCEO, oraingofof win!
RIfans picspeserer hee tha,
TOFonk? me ain ckntoty ded. bo'llll st ta d:
ELIS me hurf lal y, ma dus pe athouo
BEY:! Indy; by s afreanoo adicererupa anse tecor


The above model is the simplest model, training decreased the loss by almost 2 units.

# SELF ATTENTION

**Toy Example**

In [ ]:
torch.manual_seed(1337)
B,T,C = 4,8,2
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

The point here is that we shouldn't be able to communicate past our token/time , previous token should communicate some info to our current token, info flows from previous contextto current time step, one of the simplest way is for every single batch indepently for the tth token in a sequence we can take average of all the vectors in all the previous tokens and the current token but it looses the spacial arrangements of the tokens. but will ttry to bring it later.

In [ ]:
# We want x[b,t] = mean_{i<=t} x[b, i]
xbow = torch.zeros((B,T,C))
for b in range(B):
  for t in range(T):
    xprev = x[b,:t+1] # (t,C)
    xbow[b,t] = torch.mean(xprev,0) # effectively taking a mean of time dimension

**A trick in self attention**
its nothing but a matrix multiplication on unit lower trinagular matrix with another matrix which results in the average of elements in columns iteratively like we wanted.

In [ ]:
torch.manual_seed(42)
a = torch.tril(torch.ones(3,3))
b = torch.randint(0,10,(3,2)).float()
c = a @ b
print("a=")
print(a)
print("b=")
print(b)
print("c=")
print(c)

a=
tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
c=
tensor([[ 2.,  7.],
        [ 8., 11.],
        [14., 16.]])


In [ ]:
torch.manual_seed(42)
a = torch.tril(torch.ones(3,3))
# what if we change this a
a = a/ torch.sum(a, 1, keepdim=True)
b = torch.randint(0,10,(3,2)).float()
c = a @ b
print("a=")
print(a)
print("b=")
print(b)
print("c=")
print(c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


at row 2 we get avg of col1 till row2 and col2 till row2.

at row 3 we get avg of col1 and col2

In [ ]:
#Version 2
wei = torch.tril(torch.ones(T,T))
wei = wei/ wei.sum(1, keepdim=True)
xbow2 = wei @ x # (B, T, T) @ (B, T, C) ----> (B, T, C)


In [ ]:
print(xbow)
print(xbow2)

tensor([[[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199],
         [ 0.3504, -0.2238],
         [ 0.3525,  0.0545],
         [ 0.0688, -0.0396],
         [ 0.0927, -0.0682],
         [-0.0341,  0.1332]],

        [[ 1.3488, -0.1396],
         [ 0.8173,  0.4127],
         [-0.1342,  0.4395],
         [ 0.2711,  0.4774],
         [ 0.2421,  0.0694],
         [ 0.0084,  0.0020],
         [ 0.0712, -0.1128],
         [ 0.2527,  0.2149]],

        [[-0.6631, -0.2513],
         [ 0.1735, -0.0649],
         [ 0.1685,  0.3348],
         [-0.1621,  0.1765],
         [-0.2312, -0.0436],
         [-0.1015, -0.2855],
         [-0.2593, -0.1630],
         [-0.3015, -0.2293]],

        [[ 1.6455, -0.8030],
         [ 1.4985, -0.5395],
         [ 0.4954,  0.3420],
         [ 1.0623, -0.1802],
         [ 1.1401, -0.4462],
         [ 1.0870, -0.4071],
         [ 1.0430, -0.1299],
         [ 1.1138, -0.1641]]])
tensor([[[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
      

In [ ]:
#Version 3: Use Softmax
tril = torch.tril(torch.ones(T,T))
wei = torch.zeros((T,T)) #affinities
wei = wei.masked_fill(tril == 0, float('-inf')) #future can't communicate with the past
wei = F.softmax(wei, dim=-1)#normalise
xbow3 = wei @ x #sum
torch.allclose(xbow2, xbow3)

True

In [ ]:
# version 4: self-attention!
torch.manual_seed(1337)
B,T,C = 4,8,32 # batch, time, channels
x = torch.randn(B,T,C)

# let's see a single Head perform self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x)   # (B, T, 16)
q = query(x) # (B, T, 16)
wei =  q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) ---> (B, T, T)

tril = torch.tril(torch.ones(T, T))
#wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)

v = value(x)
out = wei @ v
#out = wei @ x

out.shape

torch.Size([4, 8, 16])

In [ ]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
        [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
        [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
        [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],
       grad_fn=<SelectBackward0>)

In [ ]:
#For decreasing the variance to around 1
k = torch.randn(B,T,head_size)
q = torch.randn(B,T,head_size)
wei = q @ k.transpose(-2, -1) * head_size**-0.5

In [ ]:
class LayerNorm1d: # (used to be BatchNorm1d)

  def __init__(self, dim, eps=1e-5, momentum=0.1):
    self.eps = eps
    self.gamma = torch.ones(dim)
    self.beta = torch.zeros(dim)

  def __call__(self, x):
    # calculate the forward pass
    xmean = x.mean(1, keepdim=True) # batch mean
    xvar = x.var(1, keepdim=True) # batch variance
    xhat = (x - xmean) / torch.sqrt(xvar + self.eps) # normalize to unit variance
    self.out = self.gamma * xhat + self.beta
    return self.out

  def parameters(self):
    return [self.gamma, self.beta]

torch.manual_seed(1337)
module = LayerNorm1d(100)
x = torch.randn(32, 100) # batch size 32 of 100-dimensional vectors
x = module(x)
x.shape

torch.Size([32, 100])

In [ ]:
x[:,0].mean(), x[:,0].std() # mean,std of one feature across all batch inputs

(tensor(0.1469), tensor(0.8803))

In [ ]:
x[0,:].mean(), x[0,:].std() # mean,std of a single input from the batch, of its features

(tensor(-9.5367e-09), tensor(1.0000))

#Lets Build our code based on the things we learnt

Lets also improve the model we use to for better results as per the paper **Attention Is All You Need.**

**Modules to import**

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F

**Hyperparameters**

In [ ]:
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 5000
learning_rate = 1e-3

Some new hyperparameters

In [ ]:
eval_interval = 100
eval_iters = 200
n_embd = 64 # no. of embeddings
n_head = 4 # no heads for multi head
n_layer = 4 #no. of layers
dropout = 0.0 #dropout for droping some weights during training

Here we are going to add a new variable called device , so that we can run our code on gpu

In [ ]:
if torch.cuda.is_available():
  device = 'cuda'
else:
  device = 'cpu'

After doing the above we have to pass the data we calculate to the device so that our gpu can perform calculations

In [ ]:
torch.manual_seed(1337) # for same random values every time we run

In [ ]:
# input.txt file we downloaded from Andrej Karpathy
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [ ]:
# our encoder decoder we built
string_to_integer = {ch:i for i,ch in enumerate(characters)}  # characters: corresponding integer/index  pairs for all characters
integer_to_string = {i:ch for i,ch in enumerate(characters)}  #  integer/index: corresponding characters  pairs for all characters

def encode(string):
  list = []
  for char in string:
    list.append(string_to_integer[char])
  return list

def decode(list):
  string = ''
  for element in list:
    string = string + (integer_to_string[element])
  return string

In [ ]:
# Splitting our data for training and validation 90/10 % split
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

**Data Loading**

In [ ]:
# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)  # device variable we built at hyperparameters section
    return x, y

#Mutli Head Self-Attention

we can use our bigram language model here to build a model , but we can optimise it further using **Multi head self attention**

  **#Single Head Self-Attention**

  A mechanism where each token attends to all other tokens in the same sequence using one set of Query, Key, and Value projections to produce  context-aware representation

  we can build multi head self attention by iterating over single head self attention.

In [ ]:
#Each token computes attention once, using one set of parameters.
class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

**Multi Head Self Attention** = For loop of Single head self attention upto head_size times

In [ ]:
class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

#Transformer Block

Think of it as one layer that takes in embeddings and outputs better, context-aware embeddings.

It contains Self-attention, Feed forward and layer norm

In [ ]:
class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

We will use this Trasnformer Block in our model to get better results then our simple model we made earlier

#Better Bigram Language Model

In [ ]:
# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

0.209729 M parameters


#Optimisation

We can use the one we used to before

In [ ]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

We are going to write a simple function which average outs the loss value we get after multiple iterations, so that we can understand the values of loss consistently

In [ ]:
@torch.no_grad() #prevents spending of computational resources which would have been allocated by pythorch
def estimate_loss():
    out = {}
    model.eval() #model will be defined later when we use bigram language model
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [ ]:
for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train') # Similar to x_inputs, y_targets we used in our code

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

step 0: train loss 4.4116, val loss 4.4022
step 100: train loss 2.6568, val loss 2.6670
step 200: train loss 2.5091, val loss 2.5058
step 300: train loss 2.4197, val loss 2.4336
step 400: train loss 2.3501, val loss 2.3562
step 500: train loss 2.2963, val loss 2.3125
step 600: train loss 2.2407, val loss 2.2496
step 700: train loss 2.2054, val loss 2.2187
step 800: train loss 2.1633, val loss 2.1866
step 900: train loss 2.1241, val loss 2.1504
step 1000: train loss 2.1036, val loss 2.1306
step 1100: train loss 2.0698, val loss 2.1180
step 1200: train loss 2.0380, val loss 2.0791
step 1300: train loss 2.0248, val loss 2.0634
step 1400: train loss 1.9926, val loss 2.0359
step 1500: train loss 1.9697, val loss 2.0287
step 1600: train loss 1.9627, val loss 2.0477
step 1700: train loss 1.9403, val loss 2.0115
step 1800: train loss 1.9090, val loss 1.9941
step 1900: train loss 1.9092, val loss 1.9858
step 2000: train loss 1.8847, val loss 1.9925
step 2100: train loss 1.8724, val loss 1.9757


**Finally we can generate output**

In [ ]:
# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))



KING RICHARD II:
Shal lifest made to bub, to take Our my dagatants:
Whith foul his vetward that a endrer, my fears' to zorm heavens,
Oof it heart my would but
With ensengmin latest in ov the doest not.

WARWICK:
Welll now, and thus quechiry: there's speak you love.
In Bodiet, and whom the sclittle
Enout-now what evily well most rive with is compon to the me
Town danters, If so;
Ange to shall do aleous, for dear?

KING HENRY VI:
Hark, but a
ards bring Edward?

GROKE:
As is no Rurnts I am you! who neet.
Pom mary thou contrantym so a thense.

QUEEN VINCENTIO:
O, sir, may in God't well ow, whom confessy.
Which migh.

ARCHILINIUS:
Dithul seaze Peed me: very it passce of's cruport;
How what make you fear tals: there loves
Tunkistren in deed, is xment.

CORIONIUS:
What comforts me. I with self From the walt I?

GRINION:
Which ushold.

KING HENRY Gindner:
Withrief I doot, is onter now.

Securming:
Intande whose no crown some Eiverely marry sold;
For for me watch the
our torguet! Goy, know ou

**For Even Better Results You can  modify the hyperparameters depending on the GPU**

In [ ]:
# batch_size = 64
# block_size = 256
# max_iters = 5000
# eval_interval = 500
# learning_rate = 3e-4
# device = 'cuda' if torch.cuda.is_available() else 'cpu'
# eval_iters = 200
# n_embd = 384
# n_head = 6
# n_layer = 6
# dropout = 0.2